<a href="https://colab.research.google.com/github/gusteivos/IJCA-final-challenge/blob/main/IJCA-final-challenge-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:

!pip install pandas google-adk google-generativeai -q


In [18]:

import re


In [19]:

import google.generativeai as genai
from google.adk.runners import Runner
from google.genai.types import Content, Part


In [20]:
from google.colab import userdata
import os


agent_grupo4_api_key = userdata.get("IJCA-final-challenge-agent-grupo4")
genai.configure(api_key = agent_grupo4_api_key)

os.environ["GOOGLE_API_KEY"] = agent_grupo4_api_key


In [41]:

# https://google.github.io/adk-docs/tools-custom/function-tools/#example


from typing import Optional
import pandas as pd


def get_datatran_csv(
    start_date: Optional[str] = None,
    start_time: Optional[str] = None,
    end_date: Optional[str] = None,
    end_time: Optional[str] = None,
    head: Optional[int] = None
    ):
    """
    Retorna um CSV filtrado da tabela `datatran`.

    Args:
        start_date (str|None): Filtra registros com data >= start_date (formato YYYY-MM-DD).
        start_time (str|None): Filtra registros com hora >= start_time (formato HH:MM:SS).
        end_date   (str|None): Filtra registros com data <= end_date.
        end_time   (str|None): Filtra registros com hora <= end_time.
        head       (int|None): Se definido, limita o DataFrame aos primeiros `head` registros.

    Returns:
        str: Conteúdo CSV filtrado (sem índice).
    """

    df_datatran = pd.read_csv("medallion/silver/datatran.csv")

    if start_date is not None:
        df_datatran = df_datatran[df_datatran["data"] >= start_date]

    if start_time is not None:
        df_datatran = df_datatran[df_datatran["hora"] >= start_time]

    if end_date is not None:
        df_datatran = df_datatran[df_datatran["data"] <= end_date]

    if end_time is not None:
        df_datatran = df_datatran[df_datatran["hora"] <= end_time]

    if head is not None:
        df_datatran = df_datatran.head(head)

    return df_datatran.to_csv(index=False)


In [46]:
from google.adk.agents import Agent
from google.adk.tools import FunctionTool


def create_agent_grupo4():
  return Agent(
      name="IJCAfinalchallengeagentgrupo4",
      model="gemini-2.5-flash",
      instruction="""
Você é um agente especializado em análise de dados de acidentes de trânsito utilizando os conjuntos de dados disponibilizados pela Polícia Rodoviária Federal (PRF) do Brasil.
Você possui acesso à ferramenta get_datatran_csv, que permite consultar, filtrar e recuperar dados estruturados sobre acidentes (incluindo datas, horários, localizações, gravidade, veículos e outras variáveis relevantes).
Seu objetivo é responder perguntas com base nesses dados, sempre que possível utilizando a ferramenta para obter informações atualizadas diretamente do dataset.
Quando necessário, você também pode realizar análises estatísticas, sumarizações, comparações, explicações e visualizações conceituais.
Caso uma pergunta exija dados reais do conjunto, sempre chame a tool get_datatran_csv com os parâmetros adequados.
Caso a pergunta seja conceitual ou explicativa, você pode responder diretamente.
      """,
      tools=[get_datatran_csv]
  )

agent_grupo4 = create_agent_grupo4()

In [43]:
import asyncio
from google.adk.sessions import InMemorySessionService, Session
from IPython.display import display, Markdown


# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()


my_user_id = "IJCA-final-challenge-agent-test-user"

In [47]:

async def run_agent_grupo4():

  my_session = await session_service.create_session(
    app_name=agent_grupo4.name,
    user_id=my_user_id
    )

  query = "Quantos acidentes ocorreram no Brasil no dia 2025-01-15 entre 14:00 e 18:00?" # @param {type: "string", placeholder: "Converse com o agente do grupo 4."}

  await run_agent_query(agent_grupo4, query, my_session, my_user_id)


In [48]:
await run_agent_grupo4()


🚀 Running query for agent: 'IJCAfinalchallengeagentgrupo4' in session: '42ee1f8b-4f25-4ba8-8c0f-99585515801a'...


EVENT: content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'end_date': '2025-01-15',
          'end_time': '18:00:00',
          'start_date': '2025-01-15',
          'start_time': '14:00:00'
        },
        id='adk-1ba64ba8-784b-4fb5-87f6-f852d63ef3b9',
        name='get_datatran_csv'
      ),
      thought_signature=b'\n\xf8\x03\x01\xd1\xed\x8ao\x83\x87\xdc\x13\x01J\xbdJ\xb7ye\xfb\xc3\xdcal\x9c\x04Y\xc8u\x84\xd7\x92\xb4\r\x91\xac\xa60\xe5_\x98\x15\x0cD1\x89\x1f|`\xc2:\xfc\x05qYB\xd3(Kw6\xa2\x0fc\xdfo#\x8aO\x8b\x8c\x0fH\xcc\xe4\xd8\xf4\xf0\x9eT`\xd9~\xab\x83I\xc4\xf6\xe7\xad\xb9D\xc8\xee\x13\x11\xb4...'
    ),
  ],
  role='model'
) grounding_metadata=None partial=None turn_complete=None finish_reason=<FinishReason.STOP: 'STOP'> error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=73,
  prompt_token_count=518,
  prompt_tokens_details=[
    Mod

For o dia 2025-01-15, entre 14:00 e 18:00, ocorreram 42 acidentes.

--------------------------------------------------

